### This is an example of where Scipy default optimizer (L-BFGS-B) does not correctly estimate the inefficiency variance. Even with $\gamma$ set to 0 and small measurement error, it estimates $\eta = 0$.

#### January 22, 2021 (after SFMA meeting)
Install the latest commit of `anml` from GitHub and the `logerf` branch from `SFMA`

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sfma.api import SFMAModel

## Make Simulations

In [ ]:
np.random.seed(10)

In [ ]:
n = 100

In [ ]:
intercept = 1.5
slope = 5

x_domain = [0, 10]

ineff = 0.4

sample_size_1 = [1000, 0.7]
sample_size_2 = [1000, 0.3]

In [ ]:
def frontier(x):
    return np.log(intercept + slope * x)

In [ ]:
def simulate():
    x = np.random.uniform(low=x_domain[0], high=x_domain[1], size=n)
    sample_sizes_1 = np.random.negative_binomial(
        n=sample_size_1[0], p=sample_size_1[1], size=int(n / 2)
    )
    sample_sizes_2 = np.random.negative_binomial(
        n=sample_size_2[0], p=sample_size_2[1], size=int(n / 2)
    )
    sample_sizes = np.append(sample_sizes_1, sample_sizes_2)
    
    the_frontier = frontier(x)
    inefficiency = np.random.exponential(ineff, size=n)
    means = the_frontier - inefficiency
    samples = [np.random.normal(m, scale=4, size=s) for m, s in zip(means, sample_sizes)]
    est_means = np.array([np.mean(s) for s in samples])
    est_sterr = np.array([np.sqrt(np.sum(sum((s - np.mean(s))**2)) / ((len(s) - 1)))/np.sqrt(len(s)) for s in samples])
    df = pd.DataFrame({
        'output': est_means,
        'se': est_sterr,
        'input': x,
        'ones': np.ones(len(x)),
        'frontier': the_frontier,
        'truth': means,
        'sample_size': sample_sizes
    })
    return df

In [ ]:
sim = simulate()

In [ ]:
the_frontier = sim['frontier']
linspace = np.linspace(x_domain[0], x_domain[1])
front = frontier(linspace)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(linspace, front, linestyle='solid')
ax.scatter(sim.input, sim.output, color='orange')
ax.errorbar(sim.input, sim.output, yerr=sim.se, linestyle='None')

In [ ]:
model = SFMAModel(
    df=sim,
    col_output='output',
    col_se='se',
    col_input='input',
    include_gamma=True
)
concave = SFMAModel(
    df=sim,
    col_output='output',
    col_se='se',
    col_input='input',
    r_linear=True,
    concave=True,
    include_gamma=True,
)

In [ ]:
model.fit(options={'solver_options': {}})
# concave.fit(options={'solver_options': {}})

In [ ]:
sim['base_predictions'] = model.predict()
# sim['concave_predictions'] = concave.predict()
sim.sort_values('input', inplace=True)

#### The last entry is $\eta$ and you can see that it's 0 for the `model` object but non-zero correct) for the `concave` object.

In [ ]:
model.x_init

In [ ]:
model.solver.x_opt

In [ ]:
concave.solver.x_opt

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(8, 4))
axes.plot(linspace, front, linestyle='dashed', color='black')
axes.scatter(sim.input, sim.output, color='grey', alpha=0.4, label="data")
# axes.scatter(sim.input, sim.output + concave.inefficiencies, color='#008080', alpha=0.4, label="data + inefficiency")
axes.errorbar(sim.input, sim.output, yerr=sim.se, linestyle='None', color='grey', alpha=0.4)
axes.plot(sim.input, sim.base_predictions, color='red', label='basic (L-BFGS-B)')
# axes.plot(sim.input, sim.concave_predictions, color='green', label='concave (trust-constr)')
axes.legend()
plt.savefig("results.png", bbox_inches="tight")

In [ ]:
model.solver.result

In [ ]:
p = np.random.uniform(size=8)

In [ ]:
p

In [ ]:
model.marginal_model.gradient(x=p, data=model.data)

In [ ]:
model.marginal_model.gradient_ad(x=p, data=model.data)

In [ ]:
model.data.obs.shape[0]